# Computation of splash shape


##  Documentation and Imports


Created on 23-06-2021 (Adapted from ShapeComputation.ipynb)

Author: Valentin Laplaud


In [ ]:
# plotting stuff
import matplotlib as mpl
mpl.use('TkAgg')
%matplotlib inline

COLOR = 'white'
COLOR2 = 'black'

mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['axes.edgecolor'] = COLOR

mpl.rcParams["figure.facecolor"] = COLOR2
mpl.rcParams["axes.facecolor"] = COLOR2
mpl.rcParams["savefig.facecolor"] = COLOR2

import matplotlib.pyplot as plt
from cycler import cycler


# numbers handling
import numpy as np
import pandas as pd

# signal processing 
from scipy.signal import savgol_filter, correlate, correlation_lags
from scipy.interpolate import interp1d

# images handling
from skimage import io
from skimage.filters import threshold_mean, gaussian, laplace, sobel
from skimage.measure import label, regionprops, regionprops_table
from skimage.util import invert
from skimage.morphology import binary_opening, binary_closing, remove_small_holes,binary_erosion
from skimage.color import rgb2gray, rgb2hsv
from skimage.segmentation import active_contour, morphological_geodesic_active_contour,morphological_chan_vese, checkerboard_level_set, inverse_gaussian_gradient
import cv2 as cv
import pims
import trackpy as tp

# to hide known warnings
import warnings
warnings.filterwarnings("ignore")

# General system functions
import os
import shutil
import sys

import time

# my functions
sys.path.append('../')
import VallapFunc as vf


##  Main functions
   

### Droplet tracking
Using trackpy

In [ ]:
def DropTack(P,StackList,SD,Scale,FPS):

    SD = SD.loc[StackList]
    
    TD = pd.DataFrame(data=None)
    
    if not os.path.exists(P + '\\Tracking'):
            os.mkdir(P + '\\Tracking') # create global folder 
    
    if not os.path.exists(P + '\\Trajectories'):
            os.mkdir(P + '\\Trajectories') # create global folder  
    
    for s in StackList:
        
        if not os.path.exists(P + '\\Tracking\\' + s):
            os.mkdir(P + '\\Tracking\\' + s) # create folder 
        
        if not os.path.exists(P + '\\Trajectories\\' + s):
            os.mkdir(P + '\\Trajectories\\' + s) # create folder 
        
        Stack = io.imread(P + '\\' + s + '_Processed_Cut.tif') # get the tiff stack
        
        print('Tracking for video : ' + s)
        # Drop position
        Xd = SD.loc[s,'DropXc']
        Yd = SD.loc[s,'DropYc']
        Rd = SD.loc[s,'DropDiam']/2*Scale
        # Target position
        Xt = SD.loc[s,'TargetXc']
        Yt = SD.loc[s,'TargetYc']
        Rt = SD.loc[s,'TargetDiam']/2*Scale
        
        # Tracking
        frames = pims.open(P + '\\' + s + '_Processed_Cut.tif')
        tp.quiet()
        f = tp.batch(frames, 11, invert=False, minmass=250, maxsize=10) # 15px size estimate 


        for i in range(len(frames)):
            fig, ax = plt.subplots(dpi=250)
            tp.annotate(f[f['frame']==i], frames[i])
            fig.savefig(P + '\\Tracking\\' + s + '\\' + str(i) + '.tif')
            plt.close()

        @tp.predict.predictor
        def predict(t1, particle):
            velocity = 3*(np.array(((particle.pos[0]-Xt),(particle.pos[1]-Yt)))
                          /np.sqrt(np.square(particle.pos[0]-Xt)+np.square(particle.pos[1]-Yt))) # Going away from target
            return particle.pos + velocity * (t1 - particle.t)
        
        t = tp.link(f, 10, memory=1,predictor=predict) # 5px max from the prediction, 2 images memory 
        
        
        t = tp.filter_stubs(t, 10) # remove trajectories with less than 10 images (1 ms)
        
        t['EjectionSide'] = t.apply(lambda row : 
                                    np.sqrt(np.square(row['x']-Xd)+np.square(row['y']-Yd))>
                                    np.sqrt(np.square(row['x']-Xt)+np.square(row['y']-Yt)),
                                    axis = 1)
        t = t[t['EjectionSide']==True] # Keep only trajectories going in the direction opposite to the drop

        t['targetDistance'] = t.apply(lambda row : 
                                      np.sqrt(np.square(row['x']-Xt)+np.square(row['y']-Yt)), axis = 1)
       
        particleList = np.unique(t['particle'].values)
        
        maxSpeed = 0
        maxQ = 0
        
        for p in particleList:
            trajP = t.loc[t['particle'] == p,['x','y','frame','mass','targetDistance']]
            
            X = trajP['x'].values
            Y = trajP['y'].values

            x1 = X[0]
            y1 = Y[0]
            x2 = X[len(X)-1]
            y2 = Y[len(Y)-1]

            length = vf.dist(x1,y1,x2,y2)
            sumlength = np.sum(vf.dist(X[0:len(X)-1],Y[0:len(X)-1],X[1:len(X)],Y[1:len(X)]))
            persistence = length/sumlength
            
           
            if (persistence>0.9) & ((length/Scale)>3):
                
                t.loc[t['particle'] == p,'Per_OK'] = True

                trajP.loc[trajP.index[1:],'speedFromTarget'] = np.divide(
                    trajP.diff().loc[trajP.index[1:],'targetDistance'].values,
                    trajP.loc[trajP.index[1:],'frame'].diff())*FPS/Scale/1000

                trajP.loc[trajP.index[1:],'momentumFromTarget'] = np.multiply(
                    trajP.diff().loc[trajP.index[1:],'speedFromTarget'].values,
                    trajP.loc[trajP.index[1:],'mass'])

                speedP = trajP.loc[trajP['speedFromTarget']>2,'speedFromTarget'].median()
                maxSpeed = np.nanmax([maxSpeed,speedP])

                qP = trajP.loc[trajP['speedFromTarget']>2,'momentumFromTarget'].median()
                maxQ = np.nanmax([maxQ,qP])
                
            else:
                t.loc[t['particle'] == p,'Per_OK'] = False
        
        t['Splash'] = s
        
        OffC = SD.loc[s,'OffCentering_pcDropDiam']
       
        
        TD = TD.append(t.loc[t['Per_OK'] == True,
                             ['x','y','mass','size','ecc','frame','particle','targetDistance','Splash']], 
                       ignore_index=True)
        TD.loc[TD['Splash'] == s,'OffC'] = OffC
            
        SD.loc[s,'maxSpeed'] = maxSpeed
        SD.loc[s,'maxQ'] = maxQ
        
        fig,ax = plt.subplots(dpi=300)
        absc = np.linspace(0,2*np.pi,1000)
        ax.plot(Xt,Yt,'.g',ms=2)
        ax.plot(Xt+Rt*np.cos(absc),Yt+Rt*np.sin(absc),'--g',lw=0.5)
        ax.plot(Xd,Yd,'.r',ms=2)
        ax.plot(Xd+Rd*np.cos(absc),Yd+Rd*np.sin(absc),'--r',lw=0.5)
        tp.plot_traj(t[t['Per_OK']],ax = ax)
        ax.set_aspect('equal', adjustable='box')
        fig.suptitle(s)
        fig.savefig(P + '\\Trajectories\\' + s + '.tif')
        plt.close()
        
        
        
        for i in range(len(Stack)-1):
            fig0, ax0 = plt.subplots(dpi=250)
            plt.imshow(Stack[i],cmap='gray')
            xl = ax0.get_xlim()
            yl = ax0.get_ylim()
            if not t.loc[(t['frame']<i) & (t['Per_OK'])].empty:
                tp.plot_traj(t.loc[(t['frame']<i) & (t['Per_OK'])], ax = ax0,lw=0.6)
            ax0.set_xlim(xl)
            ax0.set_ylim(yl)
            fig0.savefig(P + '\\Trajectories\\' + s + '\\' + str(i) + '.tif')
            plt.close()
        

    return(SD,TD)
        

### Plot trajectories characteristics

In [ ]:
def plotTraj(TD,SD,StackList,P,Name,Scale,FPS):
    
    if not os.path.exists(P + '\\DropSpeeds'):
            os.mkdir(P + '\\DropSpeeds') # create global folder  
            
    plotDF = pd.DataFrame(data=None)
    
    for s in StackList:
        OffC = round(SD.loc[s,'OffCentering_pcDropDiam']*10)/10
        
        t = TD[TD['Splash'] == s]
        
        
        particleList = np.unique(t['particle'].values)

        fig0,ax0 = plt.subplots(dpi = 250)
        plt.xlabel('Frame')
        plt.ylabel('Speed from target (m/s)')
        fig0.suptitle(s)
        
        for p in particleList:
            trajP = t.loc[t['particle'] == p,['x','y','frame','mass','size','targetDistance']]
            trajP.loc[trajP.index[1:],'speedFromTarget'] = np.divide(
                        trajP.diff().loc[trajP.index[1:],'targetDistance'].values,
                        trajP.loc[trajP.index[1:],'frame'].diff())*FPS/Scale/1000
            trajP.plot(x = 'frame', y = 'speedFromTarget', ax = ax0, legend=False)
            
            trajdata = {'particle': p,
                        'medianvel':  trajP['speedFromTarget'].median(),
                        'OffC': OffC,             
                        'size': trajP['size'].median()/Scale*1000,
                       }
            
            plotDF = plotDF.append(pd.DataFrame(data=trajdata,index = [s]))

        fig0.savefig(P + '\\DropSpeeds\\' + s + '.tif')
        plt.close()
    
    fig1,ax1 = plt.subplots(dpi=400)
    boxprops = dict(linestyle='-', linewidth=1, color='darkgreen')
    medianprops = dict(linestyle='-', linewidth=1, color='firebrick')
    flierprops = dict(marker='o', markerfacecolor='r', markersize=6,markeredgecolor='k', linewidth=0.5)
    whiskerprops = dict(linestyle='-', linewidth=1, color='darkgreen')
    plotDF.boxplot(column= 'medianvel', by = 'OffC', ax= ax1, rot = 45, 
                   boxprops = boxprops, medianprops = medianprops, flierprops = flierprops, whiskerprops = whiskerprops)
    plt.ylabel('Median particle velocity (m/s)')
    plt.xlabel('Drop offcentering (% of Diam.)')
    ax1.set_title(Name)
    fig1.suptitle('')    
    fig1.savefig(P + '\\DropSpeeds\\' + 'MedVelVsOffCent_' + Name + '.tif')
    plt.show()
    
    fig11,ax11 = plt.subplots(dpi=400)
    boxprops = dict(linestyle='-', linewidth=1, color='darkgreen')
    medianprops = dict(linestyle='-', linewidth=1, color='firebrick')
    flierprops = dict(marker='o', markerfacecolor='r', markersize=6,markeredgecolor='k', linewidth=0.5)
    whiskerprops = dict(linestyle='-', linewidth=1, color='darkgreen')
    plotDF[plotDF['size']>250].boxplot(column= 'medianvel', by = 'OffC', ax= ax11, rot = 45, 
                   boxprops = boxprops, medianprops = medianprops, flierprops = flierprops, whiskerprops = whiskerprops)
    plt.ylabel('Median particle velocity (m/s)')
    plt.xlabel('Drop offcentering (% of Diam.)')
    ax11.set_title(Name + ' - size > 250')
    fig11.suptitle('')    
    fig11.savefig(P + '\\DropSpeeds\\' + 'MedVelVsOffCent_size250_' + Name + '.tif')
    plt.show()
    
    cutoff = 50
    fig2, [ax21,ax22] = plt.subplots(ncols = 2, dpi=400)
    plotDF[plotDF['OffC']<50].hist(column = 'size',bins=np.linspace(150,400,13), ax = ax21,edgecolor = 'k',grid=False)
    plotDF[plotDF['OffC']>50].hist(column = 'size',bins=np.linspace(150,400,13), ax = ax22,edgecolor = 'k',grid=False)
    ax21.set_xlabel('Particle radius of giration (µm)')
    ax22.set_xlabel('Particle radius of giration (µm)')
    ax21.set_ylabel('Count')
    ax22.set_ylabel('Count')
    ax21.set_title('Offcent < ' + str(cutoff) + '%')
    ax22.set_title('Offcent > ' + str(cutoff) + '%')
    ax21.set_xticks(np.linspace(150,400,6))
    ax22.set_xticks(np.linspace(150,400,6))
    fig2.suptitle(Name)
    fig2.savefig(P + '\\DropSpeeds\\' + 'SizeDist_' + Name + '.tif')
    plt.show()
    
    fig3, ax3 = plt.subplots(dpi=400)
    plotDF.plot.scatter(x = 'size', y = 'medianvel', ax = ax3)
    plt.xlabel('Particle radius of giration (µm)')
    plt.ylabel('Median particle velocity (m/s)')
    ax1.set_title(Name)
    fig3.savefig(P + '\\DropSpeeds\\' + 'MedVelVsSize_' + Name + '.tif')
    plt.show()
    
    fig4,ax4 = plt.subplots(dpi=400)
    boxprops = dict(linestyle='-', linewidth=1, color='c')
    medianprops = dict(linestyle='-', linewidth=1, color='firebrick')
    flierprops = dict(marker='o', markerfacecolor='m', markersize=6,markeredgecolor='k', linewidth=0.5)
    whiskerprops = dict(linestyle='-', linewidth=1, color='c')
    plotDF.boxplot(column= 'size', by = 'OffC', ax= ax4, rot = 45, 
                   boxprops = boxprops, medianprops = medianprops, flierprops = flierprops, whiskerprops = whiskerprops)
    plt.ylabel('Median particle size (µm)')
    plt.xlabel('Drop offcentering (% of Diam.)')
    ax4.set_title(Name)
    fig4.suptitle('')    
    fig4.savefig(P + '\\DropSpeeds\\' + 'MedSizVsOffCent_' + Name + '.tif')
    plt.show()

## Enter data and run



### Data details

In [ ]:
###### Experiment of the 21-06-2021, P/C-CO-D3 with lab prints from the 14/04, 3mm diam, 15mm height
# Spatial scale (px/mm)
Scale210621 = 11.34  
# Frames per second
FPS210621 = 10000 
# Paths to data
P210621 = r'D:\Users\laplaud\Desktop\PostDoc\Data\SplashCups\2021.06.21'
# List of videos to analyze
StackList210621 = ['CC0_D3_1','CC0_D3_2','CC0_D3_3','CC0_D3_4','CC0_D3_5',
                  'CC0_D3_6','CC0_D3_7','CC0_D3_8','CC0_D3_9','CC0_D3_10',
                  'PC0_D3_1','PC0_D3_2','PC0_D3_3','PC0_D3_4','PC0_D3_5',
                  'PC0_D3_6','PC0_D3_7','PC0_D3_8','PC0_D3_9','PC0_D3_10'] 

StackList210621_PC0 = ['PC0_D3_1','PC0_D3_2','PC0_D3_3','PC0_D3_4','PC0_D3_5',
                       'PC0_D3_6','PC0_D3_7','PC0_D3_8','PC0_D3_9','PC0_D3_10'] 
#StackList210621_PC0 = ['PC0_D3_1'] 

StackList210621_CC0 = ['CC0_D3_1','CC0_D3_2','CC0_D3_3','CC0_D3_4','CC0_D3_5',
                  'CC0_D3_6','CC0_D3_7','CC0_D3_8','CC0_D3_9','CC0_D3_10'] 
#StackList210621_CC0 = ['CC0_D3_2'] 

###### Experiment of the 02-07-2021, P/C-CO-D3 with lab prints from the 14/04, 3mm diam, 12mm height
# Spatial scale (px/mm)
Scale210702 = 11.20 
# Frames per second
FPS210702 = 10000 
# Paths to data
P210702 = r'D:\Users\laplaud\Desktop\PostDoc\Data\SplashCups\2021.07.02'
# List of videos to analyze
StackList210702_PC0 = ['PC0_13','PC0_14','PC0_15','PC0_16','PC0_17','PC0_18','PC0_19','PC0_20',
                       'PC0_21','PC0_22']
#'PC0_1','PC0_2','PC0_3','PC0_4','PC0_5','PC0_6','PC0_7','PC0_8','PC0_9','PC0_10',
#                      'PC0_11','PC0_12',
StackList210702_CC0 = ['CC0_24','CC0_25']
#'CC0_1','CC0_2','CC0_3','CC0_4','CC0_6','CC0_7','CC0_8','CC0_9','CC0_10', 'CC0_11','CC0_12','CC0_13','CC0_15','CC0_16','CC0_17','CC0_18','CC0_19','CC0_20','CC0_21','CC0_22'
print('Data choice made.')


### Loading data

In [ ]:
SplashData210621 = pd.read_csv(P210621 + '\\SplashData210621_Circles.csv',index_col = 'Ind')
SplashData210702 = pd.read_csv(P210702 + '\\SplashData210702_Circles.csv',index_col = 'Ind')

### Track droplets

In [ ]:
###### Droplet tracking

### Pillars
#SplashData210621_Track_PC0, TrajData210621_PC0 = DropTack(P210621,StackList210621_PC0,SplashData210621,Scale210621,FPS210621)

#SplashData210702_Track_PC0, TrajData210702_PC0 = DropTack(P210702,StackList210702_PC0,SplashData210702,Scale210702,FPS210702)


### Cups
#SplashData210621_Track_CC0, TrajData210621_CC0 = DropTack(P210621,StackList210621_CC0,SplashData210621,Scale210621,FPS210621)

SplashData210702_Track_CC0, TrajData210702_CC0 = DropTack(P210702,StackList210702_CC0,SplashData210702,Scale210702,FPS210702)


### Save Data

In [ ]:
## Saving
SplashData210621_Track_PC0.to_csv(P210621 + '\\SplashData210621_Track_PC0.csv',index_label = 'Ind')
TrajData210621_PC0.to_csv(P210621 + '\\TrajData210621_PC0.csv',index_label = 'Ind')

SplashData210702_Track_PC0.to_csv(P210702 + '\\SplashData210702_Track_PC0.csv',index_label = 'Ind')
TrajData210702_PC0.to_csv(P210702 + '\\TrajData210702_PC0.csv',index_label = 'Ind')

SplashData210621_Track_CC0.to_csv(P210621 + '\\SplashData210621_Track_CC0.csv',index_label = 'Ind')
TrajData210621_CC0.to_csv(P210621 + '\\TrajData210621_CC0.csv',index_label = 'Ind')

SplashData210702_Track_CC0.to_csv(P210702 + '\\SplashData210702_Track_CC0.csv',index_label = 'Ind')
TrajData210702_CC0.to_csv(P210702 + '\\TrajData210702_CC0.csv',index_label = 'Ind')

### Plot results

#### General results

#### general plots
fig, ax0 = plt.subplots(dpi=250)
fig.suptitle('PC0')
SplashData210621_Track_PC0.plot.scatter(x = 'OffCentering_pcDropDiam',y = 'maxSpeed',
                                        xlabel = 'Drop offcent (% diam)',ylabel = 'Fastest droplet (m/s)', ax = ax0)
plt.show()
fig, ax0 = plt.subplots(dpi=250)
fig.suptitle('PC0')
SplashData210621_Track_PC0.plot.scatter(x = 'OffCentering_pcDropDiam',y = 'maxQ',
                                        xlabel = 'Drop offcent (% diam)',ylabel = 'Max momentum (px*m/s)', ax = ax0)
plt.show()

fig, ax0 = plt.subplots(dpi=250)
fig.suptitle('CC0')
SplashData210621_Track_CC0.plot.scatter(x = 'OffCentering_pcDropDiam',y = 'maxSpeed',
                                        xlabel = 'Drop offcent (% diam)',ylabel = 'Fastest droplet (m/s)', ax = ax0)
plt.show()
fig, ax0 = plt.subplots(dpi=250)
fig.suptitle('CC0')
SplashData210621_Track_CC0.plot.scatter(x = 'OffCentering_pcDropDiam',y = 'maxQ',
                                        xlabel = 'Drop offcent (% diam)',ylabel = 'Max momentum (px*m/s)', ax = ax0)
plt.show()

#### Trajectory characteristics

In [ ]:
# Trajectory plots

plotTraj(TrajData210621_PC0,SplashData210621_Track_PC0,StackList210621_PC0,P210621,'210621_PC0',Scale210621,FPS210621)

plotTraj(TrajData210702_PC0,SplashData210702_Track_PC0,StackList210702_PC0,P210702,'210702_PC0',Scale210702,FPS210702)

plotTraj(TrajData210621_CC0,SplashData210621_Track_CC0,StackList210621_CC0,P210621,'210621_CC0',Scale210621,FPS210621)

plotTraj(TrajData210702_CC0,SplashData210702_Track_CC0,StackList210702_CC0,P210702,'210702_CC0',Scale210702,FPS210702)


## Test Zone